Hyperparameter tuning

AUtohyperparameter tuning Algo=>GridsearchCV,randonCV and Bayerion search

Bayerian serach (Optuna library)

!pip install optuna


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 16.1 MB/s eta 0:00:00


In [ ]:
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler


In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/mtalibfarooq/Machine_Learning_Diabetes_Dataset/main/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
#replacing missing values with NaN in clumns with zero
df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']]=df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.nan)
# imputing the NaN values with mean of corresponding columns
df.fillna(df.mean(),inplace=True)

#crosching still any null values
print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [ ]:
#spliting features columns (X) and target (Y)
X=df.drop('Outcome',axis=1)
Y=df['Outcome']

#spliting data into train test
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

#appling standardization
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


#shape of data
print(X_train.shape,X_test.shape)
print(Y_train.shape,Y_test.shape)

(614, 8) (154, 8)
(614,) (154,)


In [ ]:
from sklearn.model_selection import cross_val_score

#define the objective fuction

def objective(trial):
  #suggesting the values for hyperparameter
  n_estimators=trial.suggest_int('n_estimators', 50, 200)
  max_depth=trial.suggest_int('max_depth', 3, 20)

  #creating Randomforest
  model=RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth, random_state=42)

  #applying 3 fold cross-validation and calculation
  score=cross_val_score(model,X_train,Y_train,cv=3,scoring='accuracy').mean()

  return score

  #training

In [ ]:
# created a study and optimize the objective fuction
study=optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler()) #aim to get max accuracy
study.optimize(objective,n_trials=50) # runs 50 trials to find best hyperparameter

[I 2025-08-20 17:09:04,149] A new study created in memory with name: no-name-1d8c5728-ab1a-4bd3-9e3f-39051377d1ff
[I 2025-08-20 17:09:04,803] Trial 0 finished with value: 0.771975131516021 and parameters: {'n_estimators': 91, 'max_depth': 15}. Best is trial 0 with value: 0.771975131516021.
[I 2025-08-20 17:09:05,212] Trial 1 finished with value: 0.7540490993145226 and parameters: {'n_estimators': 70, 'max_depth': 3}. Best is trial 0 with value: 0.771975131516021.
[I 2025-08-20 17:09:05,871] Trial 2 finished with value: 0.7687151283277539 and parameters: {'n_estimators': 94, 'max_depth': 8}. Best is trial 0 with value: 0.771975131516021.
[I 2025-08-20 17:09:06,180] Trial 3 finished with value: 0.7540570699824646 and parameters: {'n_estimators': 51, 'max_depth': 3}. Best is trial 0 with value: 0.771975131516021.
[I 2025-08-20 17:09:06,706] Trial 4 finished with value: 0.7736091184441256 and parameters: {'n_estimators': 70, 'max_depth': 19}. Best is trial 4 with value: 0.7736091184441256.

1. we can use multiple sampler like (Randomsampler,  gridserach)

In [ ]:
#print the best results
print(f'Best Accuracy: {study.best_value}')
print(f'Best Hyperparameters: {study.best_params}')

Best Accuracy: 0.7882751474573569
Best Hyperparameters: {'n_estimators': 110, 'max_depth': 13}


In [ ]:
from sklearn.metrics import accuracy_score

#now train a model using best parameters
best_model=RandomForestClassifier(n_estimators=study.best_params['n_estimators'],
                                  max_depth=study.best_params['max_depth'],
                                  random_state=42)

#fit the model for traing set
best_model.fit(X_train,Y_train)

#make prediction
y_pred=best_model.predict(X_test)

#calculate accuracy
accuracy=accuracy_score(Y_test,y_pred)
print(f'Accuracy: {accuracy}')


Accuracy: 0.7532467532467533


In [ ]:
# visulaization
from optuna.visualization import plot_optimization_history,plot_param_importances,plot_parallel_coordinate,plot_slice,plot_contour


In [ ]:
#1 optimizer histry plot
plot_optimization_history(study)

In [ ]:
#2. Parallel Coordination plot
plot_parallel_coordinate(study)

In [ ]:
#3.slice plot
plot_slice(study)

In [ ]:
#4.Contour plot
plot_contour(study)

In [ ]:
#5. feature importance
plot_param_importances(study)

## Optimizing Multiple ML Models

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC


In [ ]:
#define the objective fucction for Optuna
def objective(trial):
  #choose the alorithm
  classifier_algo = trial.suggest_categorical('classifier', ['RandomForest', 'SVC', 'AdaBoost', 'GradientBoosting'])

  if classifier_algo == 'RandomForest':
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    booststrap=trial.suggest_categorical('booststrap',['True','False'])

    model=RandomForestClassifier(n_estimators=n_estimators,
                                 max_depth=max_depth,
                                 min_samples_split=min_samples_split,
                                 min_samples_leaf=min_samples_leaf,
                                 bootstrap=booststrap,random_state=42)

  elif classifier_algo == 'SVC':
    C = trial.suggest_float('C', 0.1, 10)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf'])
    gamma=trial.suggest_categorical('gamma',['scale','auto'])

    model=SVC(C=C,kernel=kernel,gamma=gamma,random_state=42)

  elif classifier_algo == 'AdaBoost':
      n_estimators = trial.suggest_int('n_estimators', 50, 200)
      learning_rate = trial.suggest_float('learning_rate', 0.01, 1.0)

      model=AdaBoostClassifier(n_estimators=n_estimators,learning_rate=learning_rate,random_state=42)

  elif classifier_algo == 'GradientBoosting':
      n_estimators = trial.suggest_int('n_estimators', 50, 200)
      learning_rate = trial.suggest_float('learning_rate', 0.01, 1.0)
      max_depth = trial.suggest_int('max_depth', 3, 10)
      min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
      min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)


      model=GradientBoostingClassifier(n_estimators=n_estimators,max_depth=max_depth,min_samples_split=min_samples_split,
                                           min_samples_leaf=min_samples_leaf,learning_rate=learning_rate,random_state=42)


  # model performance cross-validation
  score=cross_val_score(model,X_train,Y_train,cv=3,scoring='accuracy').mean()

  return score

In [ ]:
!pip install cmaes

In [ ]:
#creating a study and optimizer it using CamEs sampler
study=optuna.create_study(direction='maximize', sampler=optuna.samplers.CmaEsSampler())
study.optimize(objective,n_trials=100)

[I 2025-08-20 17:53:09,467] A new study created in memory with name: no-name-d213d755-f7ff-411b-968c-ba82053f440e
[I 2025-08-20 17:53:12,623] Trial 0 finished with value: 0.7540172166427547 and parameters: {'classifier': 'AdaBoost', 'n_estimators': 97, 'learning_rate': 0.5136243712937351}. Best is trial 0 with value: 0.7540172166427547.
[W 2025-08-20 17:53:12,690] The parameter `classifier` in Trial#1 is sampled independently using `RandomSampler` instead of `CmaEsSampler`, potentially degrading the optimization performance. This fallback happend because dynamic search space and `CategoricalDistribution` are not supported by `CmaEsSampler`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler` if this independent sampling is intended behavior.
[I 2025-08-20 17:53:17,474] Trial 1 finished with value: 0.7572931611669058 and parameters: {'classifier': 'AdaBoost', 'n_estimators': 175, 'learning_rate': 0.7319439510535228}. Best 

In [ ]:
#retrival the best trials
print(f'Best Accuracy: {study.best_value}')
print(f'Best Hyperparameters: {study.best_params}')

Best Accuracy: 0.7849992029332058
Best Hyperparameters: {'classifier': 'RandomForest', 'n_estimators': 55, 'max_depth': 14, 'min_samples_split': 9, 'min_samples_leaf': 2, 'bootstrap': True}


In [ ]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,system_attrs_cma:generation,state
0,0,0.754017,2025-08-20 17:53:09.468779,2025-08-20 17:53:12.623748,0 days 00:00:03.154969,NaN,NaN,AdaBoost,NaN,NaN,0.513624,NaN,NaN,NaN,97.0,NaN,COMPLETE
1,1,0.757293,2025-08-20 17:53:12.631275,2025-08-20 17:53:17.474223,0 days 00:00:04.842948,NaN,NaN,AdaBoost,NaN,NaN,0.731944,NaN,NaN,NaN,175.0,0.0,COMPLETE
2,2,0.742667,2025-08-20 17:53:17.475845,2025-08-20 17:53:19.201320,0 days 00:00:01.725475,NaN,NaN,GradientBoosting,NaN,NaN,0.503090,9.0,9.0,10.0,130.0,0.0,COMPLETE
3,3,0.758943,2025-08-20 17:53:19.202402,2025-08-20 17:53:19.273329,0 days 00:00:00.070927,0.943326,NaN,SVC,auto,rbf,NaN,NaN,NaN,NaN,NaN,0.0,COMPLETE
4,4,0.749203,2025-08-20 17:53:19.274493,2025-08-20 17:53:20.100782,0 days 00:00:00.826289,NaN,NaN,GradientBoosting,NaN,NaN,0.892367,4.0,3.0,2.0,90.0,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.760561,2025-08-20 17:55:15.477904,2025-08-20 17:55:16.931660,0 days 00:00:01.453756,NaN,True,RandomForest,NaN,NaN,NaN,19.0,10.0,3.0,144.0,NaN,COMPLETE
96,96,0.745943,2025-08-20 17:55:16.933366,2025-08-20 17:55:19.247458,0 days 00:00:02.314092,NaN,NaN,GradientBoosting,NaN,NaN,0.756693,9.0,10.0,8.0,133.0,NaN,COMPLETE
97,97,0.750789,2025-08-20 17:55:19.248612,2025-08-20 17:55:22.667964,0 days 00:00:03.419352,NaN,NaN,AdaBoost,NaN,NaN,0.246079,NaN,NaN,NaN,160.0,NaN,COMPLETE
98,98,0.747561,2025-08-20 17:55:22.669381,2025-08-20 17:55:22.739868,0 days 00:00:00.070487,2.177849,NaN,SVC,auto,rbf,NaN,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [ ]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
AdaBoost,30
GradientBoosting,29
RandomForest,23
SVC,18


In [ ]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
AdaBoost,0.755186
GradientBoosting,0.755044
RandomForest,0.769836
SVC,0.750186
